# Solutions: Quiz 06 - Software Dev
This notebook provides reference solutions for the programming problems in the quiz. Each solution cell includes a small demonstration test to show expected behavior.

## Solution: check_line_length(path, max_len)
Reads a file line-by-line and returns a list of tuples (lineno, length) for lines longer than max_len.

In [1]:
from pathlib import Path
from typing import List, Tuple

def check_line_length(path, max_len):
    """Return list of (lineno, length) for lines longer than max_len.
    Streams file content and avoids loading whole file into memory.
    """
    out = []
    with open(path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f, start=1):
            ln = len(line.rstrip('\n'))
            if ln > max_len:
                out.append((i, ln))
    return out

# Demo test
p = Path('tmp_check_line.txt')
p.write_text('short\n' + 'x'*100 + '\n')
assert check_line_length(str(p), 79) == [(2, 100)]
p.unlink()
print('check_line_length demo passed')

check_line_length demo passed


## Solution: summarize (refactor)
Refactor the provided `summarize` by extracting helpers `valid_items` and `compute_stats`. The refactored `summarize` returns the same structure as the original.

In [4]:
# One possible solution...


def _completed_purchases(purchases):
    """
    Generator that yields only the purchases whose ``completed`` flag is True.
    """
    for p in purchases:
        if p.completed:
            yield p


def _aggregate(purchases):
    """
    Returns a tuple ``(total, count)`` for the supplied iterable of purchases.
    """
    total = 0.0
    count = 0
    for p in purchases:
        total += p.amount
        count += 1
    return total, count


def _all_customer_names(purchases):
    """
    Returns a list of the ``customer`` attribute from every purchase record.
    """
    return [p.customer for p in purchases]


def purchase_summary(purchases):
    """
    Public function kept for backward compatibility.
    Delegates the heavy lifting to the private helpers defined above.
    """
    # 1️⃣ Gather all names (unchanged by completion status)
    customers = _all_customer_names(purchases)

    # 2️⃣ Work only with the completed subset
    completed_iter = _completed_purchases(purchases)

    # 3️⃣ Compute totals/counts for the completed subset
    total, completed = _aggregate(completed_iter)

    # 4️⃣ Derive the average (guard against division by zero)
    average = total / completed if completed else 0.0

    return {
        "total_spent": total,
        "completed": completed,
        "average_per_completed": average,
        "customers": customers,
    }

## Solution: parse_user / parse_admin refactor
Extract a redundant logic used by both functions so parsing logic is centralized.

In [4]:
def _clean_str(config, key, default=''):
    val = config.get(key, default)
    return val.strip()

def _clean_int(config, key, default=-1):
    val = config.get(key, default)
    if isinstance(val, str):
        try:
            return int(val)
        except:
            return default

def parse_user(cfg):
    name = _clean_str(cfg, 'name')
    age = _clean_int(cfg, 'age', default=0)
    return {'name': name, 'age': age}

def parse_admin(cfg):
    name = _clean_str(cfg, 'name')
    level = _clean_int(cfg, 'level', default=1)
    return {'name': name, 'level': level}

# Demo test
cfg = {'name': ' Alice ', 'age': '30', 'level': '2'}
assert parse_user(cfg)['name'] == 'Alice'
assert parse_admin(cfg)['level'] == 2
print('parse_user/parse_admin demo passed')

parse_user/parse_admin demo passed


## Solution: parse_config(text)
Implement a small parser that handles key=value lines, comments, blank lines, whitespace trimming, duplicate keys (last wins), and raises ValueError for malformed lines.

In [3]:
import pytest
from datetime import datetime


def convert_type(object):
    if isinstance(object, str) and object.lower() in ["true", "false"]:
        return bool(object)

    funcs = [int, float]
    for convert in funcs:
        try:
            return convert(object)
        except:
            continue
    return object


def parse_config(text: str, convert_types: bool = False):
    result = {}
    for line in text.split("\n"):
        if not line.strip() or line.strip().startswith("#"):
            continue

        if "=" not in line:
            raise ValueError("No = in non-blank line")

        key, val = line.split("=", 1)
        if not key.strip():
            raise ValueError("Empty key")

        result[key.strip()] = (
            convert_type(val.strip()) if convert_types else val.strip()
        )

    return result


def test_parse_basic():
    text = "a=1\nb=2.5\nc=hello\n"
    assert parse_config(text) == {"a": "1", "b": "2.5", "c": "hello"}


def test_parse_comment():
    text = "a=1\n # b=2.5\nc=#hello\n"
    assert parse_config(text) == {"a": "1", "c": "#hello"}


def test_parse_empty():
    text = "a=1\n\nc=hello\n"
    assert parse_config(text) == {"a": "1", "c": "hello"}


def test_parse_multiple():
    text = "a=1\nc=goodbye\nc=hello\n"
    assert parse_config(text) == {"a": "1", "c": "hello"}


def test_parse_multi_equal():
    text = "a=1=2"
    assert parse_config(text) == {"a": "1=2"}


def test_parse_no_equal():
    text = "a=1\n  \nc  hello\n"
    with pytest.raises(ValueError):
        parse_config(text)


def test_parse_empty_key():
    text = "=abc"
    with pytest.raises(ValueError):
        parse_config(text)


def test_int_success_with_int_string():
    """A plain integer string should be returned as an int."""
    assert convert_type("42") == 42  # int succeeds, float never tried


def test_int_success_with_int_input():
    """Passing an actual int also hits the int converter."""
    assert convert_type(7) == 7


def test_float_success_after_int_failure():
    """A floating‑point literal cannot be parsed as int, so the float
    converter runs and returns a float."""
    result = convert_type("3.14")
    assert isinstance(result, float)
    assert result == 3.14


def test_float_success_with_negative_number():
    assert convert_type("-2.5") == -2.5


def test_bool_success_after_numeric_fail():
    assert convert_type("TRUE") is True


def test_bool_false_for_empty_string():
    assert convert_type("") == ""


def test_bool_true_for_literal_false():
    assert convert_type("False") is True


def test_fallback_returns_original_string():
    original = "not-a-number"
    assert convert_type(original) == original


def test_fallback_returns_custom_object():
    obj = object()
    assert convert_type(obj) is obj
